# Import Library & Set Globals

In [ ]:
# base
import os
import sys
import warnings
import logging

logging.basicConfig(level="INFO")
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

# data manipulation
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

warnings.simplefilter("ignore", pd.errors.DtypeWarning)
warnings.simplefilter("ignore", pd.errors.PerformanceWarning)
mlogger = logging.getLogger("matplotlib")
mlogger.setLevel(logging.WARNING)

# single cell
import scanpy as sc
import decoupler as dc
import liana as li

# custom
sys.path.append("../..")
from python import *

import rpy2.robjects as ro
from rpy2.robjects.conversion import localconverter

converter = get_converter()

# R_preload()
%load_ext rpy2.ipython

/mnt/DATA/home/ethung/snRNAseq_analysis/.venv/lib/python3.12/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
/mnt/DATA/home/ethung/snRNAseq_analysis/.venv/lib/python3.12/site-packages/phenograph/cluster.py:13: DeprecationWarning: Please import `spmatrix` from the `scipy.sparse` namespace; the `scipy.sparse.base` namespace is deprecated and will be removed in SciPy 2.0.0.
INFO:rpy2.situation:cffi mode is CFFI_MODE.ANY
INFO:rpy2.situation:R home found: /opt/R/4.4.1/lib/R
INFO:rpy2.situation:R library path: /opt/R/4.4.1/lib/R/lib:/usr/local/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/java-11-openjdk-amd64/lib/server
INFO:rpy2.situation:LD_LIBRARY_PATH: /opt/R/4.4.1/lib/R/lib:/usr/local/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/java-11-openjdk-amd64/lib/server
INFO:rpy2.rinterface_lib.embedded:Default options to initialize R: rpy2, --quiet, --no

In [ ]:
%matplotlib inline
study = "paper_processed_FULL"

CORES = 10
DATADIR = "../../../data"
REFDIR = "../../../references"
DOUBLETMETHODS = ["scDblFinder", "DoubletFinder", "doubletdetection", "scrublet"]

# CCC

In [ ]:
# load
savedir = os.path.join(DATADIR, "processed", study, "py", "3_annotated")
annotation = "manDoublet-seuratV3-harmony-leiden_1"
adata = sc.read_h5ad(os.path.join(savedir, f"{annotation}.h5ad"))


savedir = os.path.join(DATADIR, "processed", study, "py", "4_subclustered")

annotation = "manDoublet-seuratV3-harmony-leiden_1-macro_leiden_0.5"
adata_macros = sc.read_h5ad(os.path.join(savedir, f"{annotation}.h5ad"))

annotation = "manDoublet-seuratV3-harmony-leiden_1-fibro_leiden_0.5"
adata_fibros = sc.read_h5ad(os.path.join(savedir, f"{annotation}.h5ad"))

### All Cells

In [ ]:
# ligand & receptor
cell2cell_interactions(adata, cores=CORES)
cell_interest = ["Fibroblast", "Macrophage"]
plot_c2c(
    adata,
    "ccc_filtered",
    sources=cell_interest,
    targets=cell_interest,
    top_n=50,
    figsize=(15, 10),
)

# Save as CSV
filtered = adata.uns["ccc_filtered"].copy()
filtered_macfib = filtered[
    filtered.source.isin(cell_interest) & filtered.target.isin(cell_interest)
]
# filtered_macfib.to_csv("filtered_macro_fibro_interactions")

### Fibros & Macros

In [ ]:
ewat_macros = adata_macros[adata_macros.obs["Sample Type"].isin(["eWAT"])].copy()
ewat_fibros = adata_fibros[adata_fibros.obs["Sample Type"].isin(["eWAT"])].copy()
ewat_fms = sc.concat([ewat_fibros, ewat_macros], join="outer")

ewat_fms.raw = dc.pp.swap_layer(ewat_fms, "counts")
ewat_fms.obs["Subclusters"] = pd.concat(
    [
        "Fibroblast_" + ewat_fms.obs["leiden_fibro_group"].dropna().astype(str),
        "Macrophage_" + ewat_fms.obs["leiden_macro_group"].dropna().astype(str),
    ]
).astype("category")
cell2cell_interactions(ewat_fms, cell_group="Subclusters", key="ccc_FMs", cores=CORES)

# ewat_fms.uns["ccc_FMs_filtered"][['source', 'target', 'ligand_complex', 'receptor_complex', 'specificity_rank', 'magnitude_rank']]

In [ ]:
# dc.op.show_resources()
dc.op.resource("CellChatDB", "mouse")

In [ ]:
plot_c2c(
    ewat_fms,
    "ccc_FMs_filtered",
    top_n=200,
    sources=[f"Fibroblast_{i}" for i in np.arange(4) + 1],
    targets=["Macrophage_8"],
    figsize=(10, 20),
)

In [ ]:
plot_c2c(
    ewat_fms,
    "ccc_FMs_filtered",
    top_n=200,
    sources=[f"Fibroblast_{i}" for i in np.arange(4) + 1],
    targets=[f"Macrophage_{i}" for i in np.arange(8) + 1],
    figsize=(20, 10),
)

plot_c2c(
    ewat_fms,
    "ccc_FMs_filtered",
    top_n=200,
    sources=["Macrophage_8"],
    targets=[f"Fibroblast_{i}" for i in np.arange(4) + 1],
    figsize=(5, 20),
)

### Save/Load

In [ ]:
# save
annotation = "manDoublet-seuratV3-harmony-leiden_1"
savedir = os.path.join(DATADIR, "processed", study, "py", "3_annotated")
adata.write(os.path.join(savedir, f"{annotation}.h5ad"))

In [ ]:
# save
annotation = "manDoublet-seuratV3-harmony-leiden_1-fibros_macros"
savedir = os.path.join(DATADIR, "processed", study, "py", "4_subclustered")
adata_fms.write(os.path.join(savedir, f"{annotation}.h5ad"))

In [ ]:
# load
annotation = "manDoublet-seuratV3-harmony-leiden_1-fibros_macros"
savedir = os.path.join(DATADIR, "processed", study, "py", "4_subclustered")
adata_fms = sc.read_h5ad(os.path.join(savedir, f"{annotation}.h5ad"))